# ПРОЕКТ - на знание SQL

**Содержание:**

1. [Описание проекта](#intro)
2. [Цель исследования](#intro1)
3. [Описание данных](#intro2)
4. [Загрузка и изучение данных](#intro3)
5. [Запросы к базе данных](#intro4)
6. [Выводы](#intro5)

## Описание проекта <a id='intro'></a>

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Моя первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

## Цель исследования <a id='intro1'></a>

Проанализировать базу данных для формирования ценностного предложения для нового продукта:

- посчитаем, сколько книг вышло после 1 января 2000 года;
- для каждой книги посчитаем количество обзоров и среднюю оценку;
- пределим издательство, которое выпустило наибольшее число книг толще 50 страниц (исключим из анализа брошюры);
- определим автора с самой высокой средней оценкой книг (учитываем только книги с 50 и более оценками);
- посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Описание данных  <a id='intro2'></a>

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Загрузка и изучение данных  <a id='intro3'></a>

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Посмотрим содержание таблиц базы данных:

In [2]:
# Напишем ф-цию для просмотра общ.инф. по датасетам
def describe_cols(df_data):
    df = df_data
    values = pd.DataFrame()
    
    for col in df.columns:
        values[col] = [df[col].unique(), df[col].dtypes, df[col].nunique()]
        
    return values.transpose().sort_values(by=1, ascending=True).rename(
        {0: 'Значения', 1: 'тип данных', 2: 'Кол-во уникальных значений'}, axis=1)


#функция для вывода пустых значений и их доли в столбцах датафрейма
def nan_values(df_data):
    df = df_data
    count_missing = df.isna().sum()
    percent_missing = round(df.isna().sum() * 100 / len(df), 2)
    
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                     'num_missing': count_missing,
                                     'percent_missing': percent_missing})
    
    return missing_value_df

#функция для вывода информации о файле
def df_info (df_data):
    print('Количество строк файла:', df_data.shape[0])
    print('Количество столбцов файла:', df_data.shape[1])
    print()
    print('Информация о содержимом файла:')
    display(describe_cols(df_data))
    print()
    print('Информация о пропусках данных:')
    display(nan_values(df_data))
    print()
    print('Первые пять строк таблицы:')
    display(df_data.head())

*Таблица `books`*

In [3]:
query = """
SELECT *
FROM books;
"""
con=engine.connect()
books = pd.io.sql.read_sql(sql=text(query), con = con)
books.tail()

,book_id,author_id,title,num_pages,publication_date,publisher_id
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331
999,1000,509,Zen and the Art of Motorcycle Maintenance: An ...,540,2006-04-25,143


In [4]:
df_info(books)

Количество строк файла: 1000
Количество столбцов файла: 6

Информация о содержимом файла:


,Значения,тип данных,Кол-во уникальных значений
book_id,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",int64,1000
author_id,"[546, 465, 407, 82, 125, 257, 258, 260, 563, 4...",int64,636
num_pages,"[594, 992, 322, 541, 386, 424, 400, 448, 461, ...",int64,454
publisher_id,"[93, 336, 135, 309, 268, 116, 318, 104, 83, 36...",int64,340
title,"['Salem's Lot, 1 000 Places to See Before You ...",object,999
publication_date,"[2005-11-01, 2003-05-22, 2010-12-21, 2006-10-1...",object,618



Информация о пропусках данных:


,column_name,num_missing,percent_missing
book_id,book_id,0,0.0
author_id,author_id,0,0.0
title,title,0,0.0
num_pages,num_pages,0,0.0
publication_date,publication_date,0,0.0
publisher_id,publisher_id,0,0.0



Первые пять строк таблицы:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
# подсчёт явных дубликатов
books.duplicated().sum()

0

In [6]:
# посмотрим неявные дубликаты
books['title'].unique()

array(["'Salem's Lot", '1 000 Places to See Before You Die',
       '13 Little Blue Envelopes (Little Blue Envelope  #1)',
       '1491: New Revelations of the Americas Before Columbus', '1776',
       "1st to Die (Women's Murder Club  #1)",
       "2nd Chance (Women's Murder Club  #2)",
       "4th of July (Women's Murder Club  #4)", 'A Beautiful Mind',
       'A Bend in the Road', 'A Breath of Snow and Ashes (Outlander  #6)',
       'A Brief History of Time',
       'A Caress of Twilight (Merry Gentry  #2)',
       'A Christmas Carol and Other Christmas Writings',
       'A Clash of Kings  (A Song of Ice and Fire  #2)',
       'A Crown of Swords (The Wheel of Time  #7)',
       'A Dirty Job (Grim Reaper  #1)',
       'A Drink Before the War (Kenzie & Gennaro  #1)', 'A Fine Balance',
       'A Fistful of Charms (The Hollows  #4)',
       'A Game of You (The Sandman  #5)',
       'A Great and Terrible Beauty (Gemma Doyle #1)',
       'A Hat Full of Sky (Discworld  #32; Tiffany Aching  

В данных нет пропусков и дубликатов.

*Таблица `authors`*

In [7]:
query = """
SELECT *
FROM authors;
"""
con=engine.connect()
authors = pd.io.sql.read_sql(sql=text(query), con = con)
authors.tail()

,author_id,author
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston
635,636,Åsne Seierstad/Ingrid Christopherson


In [8]:
df_info(authors)

Количество строк файла: 636
Количество столбцов файла: 2

Информация о содержимом файла:


,Значения,тип данных,Кол-во уникальных значений
author_id,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",int64,636
author,"[A.S. Byatt, Aesop/Laura Harris/Laura Gibbs, A...",object,636



Информация о пропусках данных:


,column_name,num_missing,percent_missing
author_id,author_id,0,0.0
author,author,0,0.0



Первые пять строк таблицы:


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [9]:
# подсчёт явных дубликатов
authors.duplicated().sum()

0

In [10]:
# посмотрим неявные дубликаты
authors['author'].unique()

array(['A.S. Byatt', 'Aesop/Laura Harris/Laura Gibbs', 'Agatha Christie',
       'Alan Brennert', 'Alan Moore/David   Lloyd', 'Alan Paton',
       "Albert Camus/Justin O'Brien", 'Aldous Huxley',
       'Aldous Huxley/Christopher Hitchens',
       'Aleksandr Solzhenitsyn/H.T. Willetts', 'Alexander McCall Smith',
       'Alexander Pushkin/James E. Falen', 'Alexandre Dumas/Robin Buss',
       'Alice Hoffman', 'Alice Walker', 'Alison Bechdel',
       'Allen Ginsberg/William Carlos Williams', 'Amy Sedaris', 'Amy Tan',
       'Andy Warhol', 'Anita Diamant', 'Anita Shreve', 'Ann Brashares',
       'Ann Patchett', 'Ann Rule', 'Ann-Marie MacDonald', 'Anna Quindlen',
       'Anna Sewell', 'Anne Fadiman', 'Anne Lamott', 'Anne McCaffrey',
       'Anne Rice', 'Anonymous/N.K. Sandars', 'Anthony Bourdain',
       'Aristophanes/Sarah Ruden', 'Art Spiegelman',
       'Arthur  Miller/Christopher Bigsby',
       'Arthur Conan Doyle/Anne Perry/Sidney Paget', 'Arthur Golden',
       'Astrid Lindgren/Floren

В данных нет пропусков и дубликатов.

*Таблица `publishers`*

In [11]:
query = """
SELECT *
FROM publishers;
"""
con=engine.connect()
publishers = pd.io.sql.read_sql(sql=text(query), con = con)
publishers.tail()

,publisher_id,publisher
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling
339,340,Yearling Books


In [12]:
df_info(publishers)

Количество строк файла: 340
Количество столбцов файла: 2

Информация о содержимом файла:


,Значения,тип данных,Кол-во уникальных значений
publisher_id,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",int64,340
publisher,"[Ace, Ace Book, Ace Books, Ace Hardcover, Addi...",object,340



Информация о пропусках данных:


,column_name,num_missing,percent_missing
publisher_id,publisher_id,0,0.0
publisher,publisher,0,0.0



Первые пять строк таблицы:


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [13]:
# подсчёт явных дубликатов
publishers.duplicated().sum()

0

In [14]:
# посмотрим неявные дубликаты
publishers['publisher'].unique()

array(['Ace', 'Ace Book', 'Ace Books', 'Ace Hardcover',
       'Addison Wesley Publishing Company', 'Aladdin',
       'Aladdin Paperbacks', 'Albin Michel', 'Alfred A. Knopf',
       'Alfred A. Knopf Books for Young Readers',
       'Alfred A. Knopf Borzoi Books', 'Algonquin Books', 'Allen & Ulwin',
       'Amistad', 'Anchor', 'Anchor Books',
       'Anchor Books/Knopf Doubleday Publishing Group',
       'Andrews McMeel Publishing', 'Applewood Books', 'Arrow',
       'Arrow Books', 'Arrow Books Ltd', 'Arthur A. Levine Books',
       'Arthur A. Levine Books / Scholastic Inc.',
       'Atheneum Books for Young Readers',
       'Atheneum Books for Young Readers: Richard Jackson Books',
       'Atria Books', 'Avon', 'Avon Books', 'BALLANTINE BOOKS',
       'Back Bay Books', 'Baen Books', 'Ballantine Books',
       'Ballantine Books (NY)', 'Bantam', 'Bantam Books',
       'Bantam Books  Inc.', 'Bantam Classics',
       'Bantam Doubleday Dell (NYC)', 'Bantam Spectra', 'Bantem Press',
       '

В данных нет пропусков и дубликатов.

*Таблица `ratings`*

In [15]:
query = """
SELECT *
FROM ratings;
"""
con=engine.connect()
ratings = pd.io.sql.read_sql(sql=text(query), con = con)
ratings.tail()

,rating_id,book_id,username,rating
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2
6455,6456,1000,fharris,5


In [16]:
df_info(ratings)

Количество строк файла: 6456
Количество столбцов файла: 4

Информация о содержимом файла:


,Значения,тип данных,Кол-во уникальных значений
rating_id,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",int64,6456
book_id,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",int64,1000
rating,"[4, 2, 5, 3, 1]",int64,5
username,"[ryanfranco, grantpatricia, brandtandrea, lori...",object,160



Информация о пропусках данных:


,column_name,num_missing,percent_missing
rating_id,rating_id,0,0.0
book_id,book_id,0,0.0
username,username,0,0.0
rating,rating,0,0.0



Первые пять строк таблицы:


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [17]:
# подсчёт явных дубликатов
ratings.duplicated().sum()

0

In [18]:
# посмотрим неявные дубликаты
ratings['username'].unique()

array(['ryanfranco', 'grantpatricia', 'brandtandrea', 'lorichen',
       'mariokeller', 'johnsonamanda', 'scotttamara', 'lesliegibbs',
       'abbottjames', 'valenciaanne', 'jeromebowen', 'staylor', 'susan85',
       'npowers', 'znelson', 'thomas49', 'eallen', 'shermannatalie',
       'martinadam', 'serranoangela', 'adamsabigail', 'sfitzgerald',
       'walter49', 'jasongoodman', 'shannonsutton', 'kevin78', 'carla64',
       'robert56', 'williamsangela', 'cassandra90', 'richard89',
       'ashleymalone', 'ehall', 'tara26', 'tanya01', 'lharvey', 'ryan01',
       'laura42', 'jessica49', 'stacyfrost', 'brentpowers', 'isaiahreyes',
       'ncurry', 'amymckenzie', 'daniel19', 'amy97', 'lewisdesiree',
       'todd65', 'ulowe', 'jhill', 'patriciajohnson', 'bobbyhood',
       'odavila', 'zjohnston', 'jarvispaul', 'rmiller', 'ewerner',
       'millernicholas', 'ashleyjoshua', 'andrewfox', 'shafferrobert',
       'markdunn', 'joseph14', 'qadams', 'martinyvonne', 'nicole53',
       'sabrinabrown'

В данных нет пропусков и дубликатов.

*Таблица `reviews`*

In [19]:
query = """
SELECT *
FROM reviews;
"""
con=engine.connect()
reviews = pd.io.sql.read_sql(sql=text(query), con = con)
reviews.tail()

,review_id,book_id,username,text
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...
2792,2793,1000,jarvispaul,Republican staff bit eat material measure plan...


In [20]:
df_info(reviews)

Количество строк файла: 2793
Количество столбцов файла: 4

Информация о содержимом файла:


,Значения,тип данных,Кол-во уникальных значений
review_id,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",int64,2793
book_id,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",int64,994
username,"[brandtandrea, ryanfranco, lorichen, johnsonam...",object,160
text,[Mention society tell send professor analysis....,object,2793



Информация о пропусках данных:


,column_name,num_missing,percent_missing
review_id,review_id,0,0.0
book_id,book_id,0,0.0
username,username,0,0.0
text,text,0,0.0



Первые пять строк таблицы:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [21]:
# подсчёт явных дубликатов
reviews.duplicated().sum()

0

In [22]:
# посмотрим неявные дубликаты
reviews['username'].unique()

array(['brandtandrea', 'ryanfranco', 'lorichen', 'johnsonamanda',
       'scotttamara', 'lesliegibbs', 'valenciaanne', 'abbottjames',
       'npowers', 'staylor', 'thomas49', 'znelson', 'walter49',
       'jasongoodman', 'eallen', 'sfitzgerald', 'carla64', 'kevin78',
       'robert56', 'richard89', 'cassandra90', 'williamsangela', 'ehall',
       'lharvey', 'tanya01', 'stacyfrost', 'laura42', 'isaiahreyes',
       'daniel19', 'amy97', 'todd65', 'ulowe', 'patriciajohnson',
       'martinyvonne', 'martinadam', 'ncurry', 'nicole53', 'kingandrew',
       'wsingleton', 'wknight', 'dmiller', 'shafferrobert',
       'jennifermiller', 'robert57', 'perrypatricia', 'tara26',
       'shannonsutton', 'tnolan', 'murrayerin', 'robert00', 'carriehale',
       'dharris', 'fnguyen', 'ashleyjoshua', 'tnovak', 'fharris', 'zross',
       'annawarner', 'anthonyanderson', 'paul88', 'hollandkristin',
       'jhill', 'jeromebowen', 'rbender', 'yvonnevillarreal', 'michael57',
       'xdavis', 'wendy18', 'samue

В данных нет пропусков и дубликатов.

## Запросы к базе данных <a id='intro4'></a>

**Посчитаем сколько книг вышло после 1 января 2000 года**

In [23]:
query = """
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'::date;

"""
con=engine.connect()
ansver = pd.io.sql.read_sql(sql=text(query), con = con)
ansver

,count
0,819


После 1 января 2000 вышло 819 книг.

**Для каждой книги посчитаем количество обзоров и среднюю оценку**

In [24]:
query = """
SELECT title,
       COUNT(DISTINCT review_id) AS review_cnt,
       ROUND(AVG(rating), 2) AS rating_avg
FROM books AS b 
     LEFT JOIN ratings AS r 
     ON b.book_id = r.book_id
     LEFT JOIN reviews AS rv 
     ON b.book_id = rv.book_id
GROUP BY b.book_id
ORDER BY rating_avg DESC, 
         review_cnt DESC;

"""
con=engine.connect()
ansver = pd.io.sql.read_sql(sql=text(query), con = con)
ansver

,title,review_cnt,rating_avg
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,The Big Bad Wolf (Alex Cross #9),2,5.00
4,Welcome to Temptation (Dempseys #1),2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


В данных 1000 книг, отсортированные по средней оценке и количеству обзоров. Книга с наибольшим рейтингом и количеством обзоров (среди средних оценок 5) - A Dirty Job (Grim Reaper #1).

**Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры**

In [25]:
query = """
WITH publisher_books_cnt AS (  --определяем кол-во книг, выпущенное каждым издательством
    SELECT publisher,
           COUNT(book_id) AS books_cnt
    FROM publishers AS p
    INNER JOIN books AS b 
    ON p.publisher_id = b.publisher_id
    WHERE num_pages > 50
    GROUP BY publisher
    ORDER BY books_cnt DESC
    )
--определяем издательство с максимальным кол-вом выпущенных книг (их может быть несколько с одинаковым количеством)
SELECT publisher
FROM publisher_books_cnt
WHERE books_cnt = (SELECT MAX(books_cnt)
                   FROM publisher_books_cnt);

"""
con=engine.connect()
ansver = pd.io.sql.read_sql(sql=text(query), con = con)
ansver

,publisher
0,Penguin Books


Наибольшее количество книг (толщеной более 50 страниц) выпустило издательство *Penguin Books*.

**Определим автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками**

In [26]:
query = """

SELECT author,
       AVG(rating) AS mean_rating
FROM authors AS a
     JOIN books AS b 
     ON a.author_id = b.author_id
     JOIN ratings AS r 
     ON b.book_id = r.book_id

WHERE b.book_id IN (    --считаем книги с 50 и более оценками
                  SELECT book_id
                  FROM ratings 
                  GROUP BY book_id
                  HAVING COUNT(rating_id) >= 50
                  ORDER BY COUNT(rating_id) DESC)
GROUP BY author
ORDER BY mean_rating DESC
LIMIT 1;

"""
con=engine.connect()
ansver = pd.io.sql.read_sql(sql=text(query), con = con)
ansver

,author,mean_rating
0,J.K. Rowling/Mary GrandPré,4.287097


J.K. Rowling/Mary GrandPré- aвтор книг с самой высокой средней оценкой 4.287097 среди книг, которым поставили 50 и более оценок.

**Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.**

In [27]:
query = """
SELECT ROUND(AVG(review_cnt))
FROM (
       --считаем количество обзоров по каждому пользователю
       SELECT username, 
              COUNT(review_id) AS review_cnt
       FROM reviews 
       WHERE username IN (
                        --определяем пользователей, которые поставили больше 50 оценок
                          SELECT username
                          FROM ratings
                          GROUP BY username
                          HAVING COUNT(rating_id) > 50)
       GROUP BY  username
      ) AS rereview_count;
"""
con=engine.connect()
ansver = pd.io.sql.read_sql(sql=text(query), con = con)
ansver

,round
0,24.0


Пользователи, поставившие более 50 оценок, в среднем пишут по 24 обзора.

## Выводы <a id='intro5'></a>

Такм образом, сделав необходимые запросы к базе данных можно сказать, что:
- 819 книг вышло после 1 января 2000 г.;
- A Dirty Job (Grim Reaper #1) - книга с наибольшим рейтингом и количеством обзоров;
- издательство *Penguin Books* выпустило наибольшее количество книг толщеной более 50 страниц;
- aвтор книг с самой высокой средней оценкой - J.K. Rowling/Mary GrandPré (среди оценок >=50);
- пользователи, поставившие более 50 оценок, в среднем пишут по 24 обзора.